# Chapter 6: Triangle Rasterization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_06_triangle_rasterization.ipynb)

**Triangle rasterization** is the fundamental operation in real-time 3D graphics. Nearly all modern GPUs are optimized for rendering triangles, making it the building block for meshes, surfaces, and complex 3D models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Vec2:
    """2D Vector class for screen coordinates"""
    def __init__(self, x: float = 0.0, y: float = 0.0):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Vec2(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        return Vec2(self.x - other.x, self.y - other.y)
    
    def __mul__(self, scalar: float):
        return Vec2(self.x * scalar, self.y * scalar)
    
    def __repr__(self):
        return f"Vec2({self.x:.3f}, {self.y:.3f})"

class Framebuffer:
    """Framebuffer with color and depth"""
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.color_buffer = np.zeros((height, width, 3), dtype=np.float32)
        self.depth_buffer = np.full((height, width), float('inf'), dtype=np.float32)
    
    def clear(self, color: Tuple[float, float, float] = (0.0, 0.0, 0.0)):
        self.color_buffer[:, :] = color
        self.depth_buffer[:, :] = float('inf')
    
    def set_pixel(self, x: int, y: int, depth: float, color: Tuple[float, float, float]):
        """Set pixel with depth testing"""
        if 0 <= x < self.width and 0 <= y < self.height:
            if depth < self.depth_buffer[y, x]:
                self.depth_buffer[y, x] = depth
                self.color_buffer[y, x] = color
    
    def get_image(self) -> np.ndarray:
        return np.clip(self.color_buffer, 0, 1)

print("✓ Base classes loaded")

## 1. Barycentric Coordinates

**Problem:** How to determine if a point is inside a triangle and interpolate values across it?

**Solution:** Use barycentric coordinates.

### Definition

Any point $\mathbf{p}$ inside or on triangle $(\mathbf{v}_0, \mathbf{v}_1, \mathbf{v}_2)$ can be expressed as:

$$
\mathbf{p} = \alpha \mathbf{v}_0 + \beta \mathbf{v}_1 + \gamma \mathbf{v}_2
$$

where:
- $\alpha + \beta + \gamma = 1$
- $(\alpha, \beta, \gamma)$ are the **barycentric coordinates**
- Point is inside triangle if $\alpha, \beta, \gamma \geq 0$

### Computing Barycentric Coordinates

Using the **edge function** method:

$$
E_{ij}(\mathbf{p}) = (\mathbf{v}_j - \mathbf{v}_i) \times (\mathbf{p} - \mathbf{v}_i)
$$

For 2D points:
$$
E(x, y) = (x_j - x_i)(y - y_i) - (y_j - y_i)(x - x_i)
$$

The barycentric coordinates are:
$$
\alpha = \frac{E_{12}(\mathbf{p})}{E_{12}(\mathbf{v}_0)}, \quad
\beta = \frac{E_{20}(\mathbf{p})}{E_{20}(\mathbf{v}_1)}, \quad
\gamma = \frac{E_{01}(\mathbf{p})}{E_{01}(\mathbf{v}_2)}
$$

### Simplified 2D Formula

$$
\alpha = \frac{(y_1 - y_2)(x - x_2) + (x_2 - x_1)(y - y_2)}{(y_1 - y_2)(x_0 - x_2) + (x_2 - x_1)(y_0 - y_2)}
$$

$$
\beta = \frac{(y_2 - y_0)(x - x_2) + (x_0 - x_2)(y - y_2)}{(y_1 - y_2)(x_0 - x_2) + (x_2 - x_1)(y_0 - y_2)}
$$

$$
\gamma = 1 - \alpha - \beta
$$

In [ ]:
def barycentric(p: Tuple[float, float], 
               v0: Tuple[float, float],
               v1: Tuple[float, float],
               v2: Tuple[float, float]) -> Tuple[float, float, float]:
    """Compute barycentric coordinates of point p in triangle (v0, v1, v2)"""
    x, y = p
    x0, y0 = v0
    x1, y1 = v1
    x2, y2 = v2
    
    denom = (y1 - y2) * (x0 - x2) + (x2 - x1) * (y0 - y2)
    
    if abs(denom) < 1e-10:
        return (-1, -1, -1)  # Degenerate triangle
    
    alpha = ((y1 - y2) * (x - x2) + (x2 - x1) * (y - y2)) / denom
    beta = ((y2 - y0) * (x - x2) + (x0 - x2) * (y - y2)) / denom
    gamma = 1.0 - alpha - beta
    
    return (alpha, beta, gamma)

def is_inside_triangle(alpha: float, beta: float, gamma: float) -> bool:
    """Check if barycentric coordinates are inside triangle"""
    return alpha >= 0 and beta >= 0 and gamma >= 0

def edge_function(a: Tuple[float, float], 
                 b: Tuple[float, float],
                 c: Tuple[float, float]) -> float:
    """Edge function: positive if c is on the left of edge ab"""
    return (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])

print("✓ Barycentric coordinate functions loaded")

In [ ]:
def rasterize_triangle(fb: Framebuffer,
                      v0: Tuple[float, float, float],  # (x, y, z)
                      v1: Tuple[float, float, float],
                      v2: Tuple[float, float, float],
                      c0: Tuple[float, float, float],  # color at v0
                      c1: Tuple[float, float, float],  # color at v1
                      c2: Tuple[float, float, float]): # color at v2
    """Rasterize triangle with color and depth interpolation"""
    
    # Bounding box
    min_x = max(0, int(math.floor(min(v0[0], v1[0], v2[0]))))
    max_x = min(fb.width - 1, int(math.ceil(max(v0[0], v1[0], v2[0]))))
    min_y = max(0, int(math.floor(min(v0[1], v1[1], v2[1]))))
    max_y = min(fb.height - 1, int(math.ceil(max(v0[1], v1[1], v2[1]))))
    
    # Rasterize
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            # Pixel center
            px = x + 0.5
            py = y + 0.5
            
            # Barycentric coordinates
            alpha, beta, gamma = barycentric(
                (px, py),
                (v0[0], v0[1]),
                (v1[0], v1[1]),
                (v2[0], v2[1])
            )
            
            # Inside triangle?
            if is_inside_triangle(alpha, beta, gamma):
                # Interpolate depth
                depth = alpha * v0[2] + beta * v1[2] + gamma * v2[2]
                
                # Interpolate color
                color = (
                    alpha * c0[0] + beta * c1[0] + gamma * c2[0],
                    alpha * c0[1] + beta * c1[1] + gamma * c2[1],
                    alpha * c0[2] + beta * c1[2] + gamma * c2[2]
                )
                
                # Set pixel with depth test
                fb.set_pixel(x, y, depth, color)

print("✓ Triangle rasterization function loaded")

## Example 1: Basic Triangle Rasterization

In [ ]:
# Create framebuffer
fb = Framebuffer(400, 400)
fb.clear((0.1, 0.1, 0.1))

# Single triangle with vertex colors
v0 = (200, 100, 0.5)
v1 = (100, 300, 0.5)
v2 = (300, 300, 0.5)

c0 = (1.0, 0.0, 0.0)  # Red
c1 = (0.0, 1.0, 0.0)  # Green
c2 = (0.0, 0.0, 1.0)  # Blue

rasterize_triangle(fb, v0, v1, v2, c0, c1, c2)

# Display
plt.figure(figsize=(8, 8))
plt.imshow(fb.get_image(), origin='lower')
plt.title('Basic Triangle Rasterization with Color Interpolation')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice smooth color gradients from barycentric interpolation.")

## Example 2: Multiple Overlapping Triangles with Depth Testing

In [ ]:
# Create framebuffer
fb2 = Framebuffer(400, 400)
fb2.clear((0.0, 0.0, 0.0))

# Define multiple triangles at different depths
triangles = [
    # Triangle 1 (front, red)
    ((200, 150, 0.3), (100, 300, 0.3), (300, 300, 0.3),
     (1.0, 0.0, 0.0), (0.7, 0.0, 0.0), (0.5, 0.0, 0.0)),
    
    # Triangle 2 (middle, green)
    ((300, 100, 0.5), (150, 250, 0.5), (350, 280, 0.5),
     (0.0, 1.0, 0.0), (0.0, 0.7, 0.0), (0.0, 0.5, 0.0)),
    
    # Triangle 3 (back, blue)
    ((150, 100, 0.7), (50, 280, 0.7), (280, 250, 0.7),
     (0.0, 0.0, 1.0), (0.0, 0.0, 0.7), (0.0, 0.0, 0.5)),
]

# Rasterize all triangles (order doesn't matter due to depth testing)
for v0, v1, v2, c0, c1, c2 in triangles:
    rasterize_triangle(fb2, v0, v1, v2, c0, c1, c2)

# Display color buffer
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.imshow(fb2.get_image(), origin='lower')
ax1.set_title('Color Buffer (with depth testing)')
ax1.axis('off')

# Visualize depth buffer
depth_vis = np.where(fb2.depth_buffer == float('inf'), 1.0, fb2.depth_buffer)
ax2.imshow(depth_vis, cmap='gray', origin='lower')
ax2.set_title('Depth Buffer (darker = closer)')
ax2.axis('off')

plt.tight_layout()
plt.show()

print("Notice how depth testing correctly handles overlapping triangles.")

## Example 3: Triangle Mesh (Tetrahedron)

In [ ]:
# Create framebuffer
fb3 = Framebuffer(400, 400)
fb3.clear((0.1, 0.1, 0.15))

# Define tetrahedron vertices (3D)
vertices_3d = [
    Vec3(0, 1, 0),      # Top
    Vec3(-1, -1, 1),    # Front left
    Vec3(1, -1, 1),     # Front right
    Vec3(0, -1, -1),    # Back
]

# Simple orthographic projection to 2D
scale = 80
offset = (200, 200)
vertices_2d = []
for v in vertices_3d:
    x = int(v.x * scale + offset[0])
    y = int(v.y * scale + offset[1])
    z = v.z * 0.5 + 0.5  # Normalize depth to [0, 1]
    vertices_2d.append((x, y, z))

# Define faces (triangles)
faces = [
    (0, 1, 2),  # Front
    (0, 2, 3),  # Right
    (0, 3, 1),  # Left
    (1, 3, 2),  # Bottom
]

# Face colors
face_colors = [
    (1.0, 0.0, 0.0),  # Red
    (0.0, 1.0, 0.0),  # Green
    (0.0, 0.0, 1.0),  # Blue
    (1.0, 1.0, 0.0),  # Yellow
]

# Rasterize each face
for face_idx, (i0, i1, i2) in enumerate(faces):
    v0 = vertices_2d[i0]
    v1 = vertices_2d[i1]
    v2 = vertices_2d[i2]
    color = face_colors[face_idx]
    
    # Use same color for all vertices
    rasterize_triangle(fb3, v0, v1, v2, color, color, color)

plt.figure(figsize=(8, 8))
plt.imshow(fb3.get_image(), origin='lower')
plt.title('Tetrahedron (Triangle Mesh)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice how depth testing correctly handles face occlusion.")

## Summary

**Triangle rasterization** is the cornerstone of real-time 3D rendering:

### Key Concepts

1. **Barycentric coordinates**: Express any point inside a triangle as weighted sum of vertices
2. **Edge function**: Determine if a point is inside the triangle
3. **Attribute interpolation**: Smoothly interpolate colors, normals, textures across triangle
4. **Depth testing**: Determine visibility of overlapping triangles

### Algorithm Steps

1. Compute bounding box
2. For each pixel in bounding box:
   - Compute barycentric coordinates
   - Test if inside triangle
   - Interpolate depth and attributes
   - Depth test and update framebuffer

### Applications

Triangle rasterization is used in:
- Real-time game rendering
- CAD software visualization
- Scientific visualization
- Virtual reality
- GPU hardware acceleration

This forms the foundation for shading, texturing, and advanced rendering techniques in the following chapters.